In [1]:
import os
from datetime import datetime
import mne

%matplotlib widget

In [2]:
sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = os.path.join(sample_data_folder, 'MEG', 'sample', 'sample_audvis_raw.fif')

In [3]:
raw = mne.io.read_raw_fif(sample_data_raw_file, verbose=False)
raw.crop(tmax=60).load_data()

<Raw  |  sample_audvis_raw.fif, n_channels x n_times : 376 x 36038 (60.0 sec), ~107.0 MB, data loaded>

### Creating annotations programatically

In [4]:
my_annot = mne.Annotations(onset=[3, 5, 7], duration=[1, 0.5, 0.25], description=['AAA', 'BBB', 'CCC'])

In [5]:
raw.set_annotations(my_annot)
print(raw.annotations)

<Annotations  |  3 segments : AAA (1), BBB (1), CCC (1), orig_time : 2002-12-03 19:01:10.720100>


In [6]:
# convert meas_date (a tuple of seconds, microseconds) into a float:
meas_date = raw.info['meas_date'][0] + raw.info['meas_date'][1] / 1e6
orig_time = raw.annotations.orig_time

In [9]:
assert meas_date == orig_time

In [10]:
time_of_first_sample = raw.first_samp / raw.info['sfreq']

print(my_annot.onset + time_of_first_sample)

[45.95597083 47.95597083 49.95597083]


In [11]:
print(raw.annotations.onset)

[45.95597088 47.95597088 49.95597088]


In [12]:
time_format = '%Y-%m-%d %H:%M:%S.%f'
new_orig_time = datetime.utcfromtimestamp(meas_date + 50).strftime(time_format)
print(new_orig_time)

2002-12-03 19:02:00.720100


In [13]:
later_annot = mne.Annotations(onset=[3, 5, 7], duration=[1, 0.5, 0.25], description=['DDD', 'EEE', 'FFF'], orig_time=new_orig_time)

In [14]:
raw2 = raw.copy().set_annotations(later_annot)
print(later_annot.onset)
print(raw2.annotations.onset)

[3. 5. 7.]
[53. 55. 57.]


In [16]:
fig = raw.plot(start=2, duration=6);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
fig2 = raw2.plot(start=2, duration=6);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Annotating Raw objects interactively

In [24]:
fig.canvas.key_press_event('a')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Operations on Annotations objects

In [26]:
new_annot = mne.Annotations(onset=3.75, duration=0.75, description='AAA')
raw.set_annotations(my_annot + new_annot)
raw.plot(start=2, duration=6);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
print(raw.annotations[0])

OrderedDict([('onset', 45.955970883369446), ('duration', 1.0), ('description', 'AAA'), ('orig_time', 1038942070.7201)])


In [28]:
print(raw.annotations[:2])

<Annotations  |  2 segments : AAA (2), orig_time : 2002-12-03 19:01:10.720100>


In [29]:
print(raw.annotations[(3, 2)])

<Annotations  |  2 segments : BBB (1), CCC (1), orig_time : 2002-12-03 19:01:10.720100>


In [31]:
for ann in raw.annotations:
    desc = ann['description']
    start = ann['onset']
    end = ann['onset'] + ann['duration']
    print("'{}' goes from {} to {}".format(desc, start, end))

'AAA' goes from 45.955970883369446 to 46.955970883369446
'AAA' goes from 46.705970883369446 to 47.455970883369446
'BBB' goes from 47.955970883369446 to 48.455970883369446
'CCC' goes from 49.955970883369446 to 50.205970883369446


In [32]:
# later_annot will be changed, because we're modifying the first element of later_annot.onset directly:
later_annot.onset[0] = 99

In [33]:
# later_annot will not be changed, because later_annot[0] returns a copy before the 'onset' field is changed
later_annot[0]['onset'] = 77

In [34]:
later_annot[0]['onset']

99.0